In [1]:
import numpy as np
import pandas as pd

In [2]:
ratings_train = pd.read_csv("../Data/Delay_time_data/edge_train.csv", low_memory=False)
# ratings_test = pd.read_csv("../Data/Delay_time_data/Test/tests - 25.csv",low_memory=False)
datasets_train = ratings_train.dataset_id.unique()
model_train = ratings_train.model_id.unique()
meta_models = pd.read_csv("../Data/models_num.csv",low_memory=False)
models = meta_models.model_id.unique()

In [ ]:
data_model_train_matrix = pd.DataFrame(index=datasets_train,columns=models)
for row in ratings_train.itertuples():
    data_model_train_matrix.loc[row[1]][row[2]] = row[3]

In [ ]:
def generate_rating_matrix(n_users, n_items, min_rating, max_rating, sparsity):
    rating_matrix = np.random.rand(n_users, n_items)
    rating_matrix = min_rating + (max_rating - min_rating) * rating_matrix
    mask = np.random.rand(n_users, n_items) > sparsity
    rating_matrix[mask] = np.nan
    return pd.DataFrame(rating_matrix)

# 参数设置
n_users = 10
n_items = 5
min_rating = 0
max_rating = 1
sparsity = 0.9 # 设置评分矩阵稀疏度，取值范围 0 到 1。值越大，矩阵中的 NaN 值越多

rating_matrix = generate_rating_matrix(n_users, n_items, min_rating, max_rating, sparsity)

In [ ]:
rating_matrix

In [ ]:
from sklearn.metrics.pairwise import pairwise_kernels

In [ ]:
def create_bipartite_adjacency_matrix(rating_matrix):
    n_users, n_items = rating_matrix.shape
    adjacency_matrix = np.zeros((n_users + n_items, n_users + n_items))
    adjacency_matrix[:n_users, n_users:] = rating_matrix
    adjacency_matrix[n_users:, :n_users] = rating_matrix.T
    return adjacency_matrix

In [ ]:
def propagation_matrix(adjacency, lambda_):
    n = adjacency.shape[0]
    I = np.eye(n)
    # 将 NaN 视为 0
    adjacency = np.nan_to_num(adjacency)
    try:
        P = np.linalg.inv(I - lambda_ * adjacency)
    except np.linalg.LinAlgError:
        print("矩阵不可逆，无法计算传播矩阵")
        return None
    return P

def propagation_matrix_withWalkLength(adjacency_matrix, max_walk_length):
    adjacency_matrix = np.nan_to_num(adjacency_matrix)
    propagation_matrix = np.eye(adjacency_matrix.shape[0])
    sum_matrix = np.eye(adjacency_matrix.shape[0])

    for _ in range(max_walk_length):
        propagation_matrix = propagation_matrix @ adjacency_matrix
        sum_matrix += propagation_matrix

    return sum_matrix


In [ ]:
# 计算二分图邻接矩阵
bipartite_adjacency_matrix = create_bipartite_adjacency_matrix(data_model_train_matrix)

In [ ]:
pd.DataFrame(bipartite_adjacency_matrix)

In [ ]:
lambda_ = 0.2  # 超参数 λ，用于调整传播矩阵的强度
propagation_lambda = propagation_matrix(bipartite_adjacency_matrix, lambda_)
propagation_maxLength = propagation_matrix_withWalkLength(bipartite_adjacency_matrix, 4)

In [ ]:
pd.DataFrame(propagation_lambda)

In [ ]:
pd.DataFrame(propagation_maxLength)

In [ ]:
# 提取用户-商品传播矩阵和商品-用户传播矩阵
n_users = data_model_train_matrix.shape[0]
user_item_propagation = propagation_maxLength[:n_users, n_users:]
item_user_propagation = propagation_maxLength[n_users:, :n_users]

In [ ]:
# 计算 Random Walk Kernel
random_walk_kernel = np.dot(user_item_propagation, item_user_propagation)

In [ ]:
pd.DataFrame(random_walk_kernel)

In [ ]:
def normalize_kernel(kernel_matrix):
    diagonal_elements = np.diag(kernel_matrix)
    normalized_kernel_matrix = np.divide(kernel_matrix, np.sqrt(np.outer(diagonal_elements, diagonal_elements)))
    return normalized_kernel_matrix

In [ ]:
normalized_kernel = normalize_kernel(random_walk_kernel)
pd.DataFrame(normalized_kernel)

In [17]:
def normalize_kernel(kernel_matrix):
    diagonal_elements = np.diag(kernel_matrix)
    normalized_kernel_matrix = np.divide(kernel_matrix, np.sqrt(np.outer(diagonal_elements, diagonal_elements)))
    return normalized_kernel_matrix

In [18]:
normalized_kernel = normalize_kernel(random_walk_kernel)
pd.DataFrame(normalized_kernel)

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,1.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.999381,0.00000,0.0,0.000000
1,0.000000,1.00000,0.0,0.000000,0.0,0.0,0.000000,1.000000,0.000000,0.999962,...,1.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.99996,0.0,0.000000
2,0.000000,0.00000,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.999115,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000
3,0.000000,0.00000,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.997179
4,0.000000,0.00000,0.0,0.000000,1.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.999998,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,1.0,0.000000,0.00000,0.0,0.000000
68,0.999381,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,0.00000,0.0,0.000000
69,0.000000,0.99996,0.0,0.000000,0.0,0.0,0.000000,0.999961,0.000000,1.000000,...,0.999961,0.0,0.0,0.0,0.0,0.0,0.000000,1.00000,0.0,0.000000
70,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,1.0,0.000000
